# Data preparation

We combine the data, convert the labels for classification and apply augmentation

In [1]:
import time, json, math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

np.random.seed(0)

In [2]:
# # load full data set again to extract max pixel value
# FILES=["../thermal_raw_20210507_full/20210507_1605_3078.txt", "../thermal_raw_20210507_full/20210507_1605_C088.txt"]
# data = list()

# for FILE in FILES:
#     with open(FILE, "r") as f:
#         for line in f:
#             record = json.loads(line)
#             data.append(record)

# # convert data to pandas and do give it appropriate datatypes
# df = pd.DataFrame(data)

# # fix datatypes, drop sensor_size
# df['Timestamp'] = df['Timestamp'].astype('int32')
# df['Sensor ID'] = df['Sensor ID'].str.replace(r'Sensor_32x32_', r'')
# df['Room Temperature'] = df['Room Temperature'].astype('float32')
# df = df.drop(['Sensor size'], axis=1)

# # reset index to get it as column
# df.reset_index(level=0, inplace=True)    
            
# max_pxl_val = np.max(df['data'].apply(lambda x: np.max(x)))

max_pxl_val = 33.0

print(max_pxl_val)

33.0


In [3]:
df_felix = pd.read_json('../data/labelled_data/felix_1637156004_labels.json', orient='records')
df_omar = pd.read_json('../data/labelled_data/omar_1636927148_labels.json', orient='records')
df_tobias = pd.read_json('../data/labelled_data/tobias_1636919640_labels.json', orient='records')
df_collab = pd.read_json('../data/labelled_data/collab_1637164025_labels.json', orient='records')

In [4]:
df = pd.concat([df_felix, df_omar, df_tobias, df_collab])
df = df.drop(columns=['index', 'changed', 'Timestamp', 'Sensor ID'])
df = df.reset_index(drop=True)

# scale - do it via batch normalization!
#df['data'] = df['data'].apply(lambda x: np.array(x) / max_pxl_val)
# apply min(1.0, real_value / 33.0) also in later inference

df['data'] = df['data'].apply(lambda arr: [list(map(lambda x: max(15.0, x), line)) for line in arr])


In [5]:
# convert labels
binary_label_converter = { 
    'Human': 1,
    'Several humans': 1, 
    'Non-human heat': 0, 
    'Ambigous heat': 0,
    'No heat':  0,
}


df_binary = df.copy()

df_binary['label'] = df['label'].apply(lambda x: binary_label_converter[x])

print(df_binary.shape)
df_binary.head(n=3)

df_binary.to_json('../data/labelled_data/prepared-samples-binary.json')

(200, 4)


In [6]:
# convert labels into five classes
numeric_label_converter = { 
    'Human': 0,
    'Several humans': 1, 
    'Non-human heat': 2, 
    'Ambigous heat': 3,
    'No heat':  4,
}

df_numeric = df.copy()

df_numeric['label'] = df['label'].apply(lambda x: numeric_label_converter[x])

print(df_numeric.shape)
df_numeric.head(n=3)

df_numeric.to_json('./data/labelled_data/prepared-samples-numeric.json')

(200, 4)
